## Import several Python packages to check availability

In [6]:
import numpy
import matplotlib
import scipy
import bokeh
import pandas

In [ ]:
# These were added explicitly to environment.yml for binder.
import plotly
import nltk
import autocorrect
# import pyenchant

## Run some tests with WordNet from NLTK

In [7]:
from nltk.corpus import wordnet as wn

sets = wn.synsets('boil')
for s in sets:
    print( s.name() )
    # print( s.pos() )

print(' ')
sets = wn.synsets('jump')
for s in sets:
    print( s.name() )
    # print( s.pos() )

print(' ')
sets = wn.synsets('flow')
for s in sets:
    print( s.name() )
    # print( s.pos() )
    
print(' ')
sets = wn.synsets('gyrate')
for s in sets:
    print( s.name() )
    # print( s.pos() )

boil.n.01
boiling_point.n.01
boil.v.01
boil.v.02
boil.v.03
churn.v.02
seethe.v.02
 
jump.n.01
leap.n.02
jump.n.03
startle.n.01
jump.n.05
jump.n.06
jump.v.01
startle.v.02
jump.v.03
jump.v.04
leap_out.v.01
jump.v.06
rise.v.11
jump.v.08
derail.v.02
chute.v.01
jump.v.11
jumpstart.v.01
jump.v.13
leap.v.02
alternate.v.01
 
flow.n.01
flow.n.02
flow.n.03
flow.n.04
stream.n.04
stream.n.02
menstruation.n.01
flow.v.01
run.v.06
flow.v.03
flow.v.04
hang.v.05
flow.v.06
menstruate.v.01
 
gyrate.v.01
spin.v.01


## Run some tests with VerbNet from NLTK

In [8]:
from nltk.corpus import verbnet as vn
vn.lemmas()[0:25]

# help(vn)

[u'December',
 u'FedEx',
 u'UPS',
 u'abandon',
 u'abase',
 u'abash',
 u'abate',
 u'abbreviate',
 u'abduct',
 u'abet',
 u'abhor',
 u'abolish',
 u'abound',
 u'abrade',
 u'abridge',
 u'absolve',
 u'abstain',
 u'abstract',
 u'abuse',
 u'abut',
 u'accelerate',
 u'accept',
 u'acclaim',
 u'accompany',
 u'accrue']

In [5]:
from autocorrect import spell

spell('proposetion')

'proposition'

In [6]:
spell('proposeal')

'proposal'

In [9]:
import pyenchant

ImportError: No module named pyenchant